In [1]:
import os

if not os.getcwd().endswith('mob2crime'):
    os.chdir('..')
os.getcwd()

'/mnt/d/1ResProj/mob2crime'

In [2]:
import glob
import urllib.request


import unicodedata

import os

import time

import pandas as pd
import geopandas as gp
%matplotlib inline

# 2010 census
https://www.inegi.org.mx/programas/ccpv/2010/default.html

## Marco geoestadístico

In [8]:
folder = 'data/mexico/geography-socioeconomics/2010CensusGeography'

### download all files Urban and rural

The maps for urban and rural in detail are arranged as per localidad per folder, it takes time to download them all

In [149]:
urban_list = pd.read_csv(f'{folder}/UrbanList.csv')

In [200]:
hdr = {
    'Host': 'www.inegi.org.mx',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'DNT': '1',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive'
}

In [ ]:
url_pattern = 'http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/urbana/SHP/{ent}/SHP/{upc}_s.zip'
for _, row in urban_list.iterrows():
    path = f'{folder}/Urban/{row.upc}_s.zip'
    if os.path.exists(path):
        continue
    ent = row.entidad.strip().replace(' ','_')
    if ent == 'Ciudad_de_México':
        ent = 'Distrito_Federal'
    else:
        ent = u"".join([c for c in unicodedata.normalize('NFD', ent) if not unicodedata.combining(c)])
        
    url = url_pattern.format(ent=ent, upc=row.upc)
#     req = urllib.request.Request(url, headers=hdr)
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    data = response.read()      
    with open(path, 'wb') as f:
        f.write(data)
    print('done', row.upc)
    time.sleep(1)

In [6]:
rural_list = pd.read_csv(f'{folder}/RuralList.csv')

In [35]:
url_pattern = 'http://internet.contenidos.inegi.org.mx/contenidos/Productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/rural/SHP/{ent}/{upc}_s.zip'

for _, row in rural_list.iterrows():
    path = f'{folder}/Rural/{row.upc}_s.zip'
    if os.path.exists(path):
        continue
    ent = row.entidad.strip().replace(' ','_')
    if ent == 'Ciudad_de_México':
        ent = 'Distrito_Federal'
    else:
        ent = u"".join([c for c in unicodedata.normalize('NFD', ent) if not unicodedata.combining(c)])
        
    url = url_pattern.format(ent=ent, upc=row.upc)
#     req = urllib.request.Request(url, headers=hdr)
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    data = response.read()      
    with open(path, 'wb') as f:
        f.write(data)
    print('done', row.upc)
#     time.sleep(1)

done 702825572921
done 702825572945
done 702825572952
done 702825572969
done 702825572976
done 702825572983
done 702825572990
done 702825573003
done 702825573010
done 702825573034
done 702825573058
done 702825573065
done 702825573072
done 702825573126
done 702825573140
done 702825573102
done 702825573157
done 702825573164
done 702825573188
done 702825573195
done 702825573201
done 702825573218
done 702825573225
done 702825573249
done 702825573256
done 702825573287
done 702825573300
done 702825573355
done 702825573379
done 702825573386
done 702825573317
done 702825573324
done 702825573348
done 702825573423
done 702825573430
done 702825573447
done 702825568368
done 702825573461
done 702825573485
done 702825573508
done 702825573515
done 702825573522
done 702825573539
done 702825573546
done 702825573560
done 702825573584
done 702825573591
done 702825573607
done 702825573621
done 702825573638
done 702825572105
done 702825571436
done 702825573454
done 702825571399
done 702825573706
done 70282

done 702825582159
done 702825582173
done 702825582180
done 702825582197
done 702825582210
done 702825582227
done 702825582234
done 702825582241
done 702825582258
done 702825582265
done 702825582272
done 702825582289
done 702825582296
done 702825582302
done 702825582319
done 702825582326
done 702825582333
done 702825582340
done 702825582357
done 702825582364
done 702825582371
done 702825582388
done 702825582395
done 702825582401
done 702825582418
done 702825582425
done 702825582432
done 702825582449
done 702825582456
done 702825582630
done 702825582470
done 702825582487
done 702825582494
done 702825582500
done 702825582517
done 702825582524
done 702825582531
done 702825582623
done 702825582166
done 702825582548
done 702825582555
done 702825582562
done 702825582579
done 702825582586
done 702825582593
done 702825582609
done 702825582616


### deprecated

#### extract ageb urban as one zip

In [6]:
import zipfile

In [9]:
urban_fns = glob.glob(f'{folder}/Urban/*.zip')

In [39]:
with zipfile.ZipFile(f'{folder}/Urban_Ageb.zip', 'a', compression=zipfile.ZIP_DEFLATED) as wzf:
    existing_fns = set(wzf.namelist())

    # loop over each file
    for i, fn in enumerate(urban_fns):
        if i % 500 == 0:
            print(f'working on {i}th, {fn}')

        # open the file
        with zipfile.ZipFile(fn) as z:
            # each file could have multiple loc
            for zfn in z.namelist():
                if zfn.endswith('/'):
                    continue

                try:
                    zfn_split = zfn.split('/')
                    length = len(zfn_split)
                    loc_id_ftype_ext = zfn_split[-1]
                    loc_id = loc_id_ftype_ext[:9]
                    ftype, ext = loc_id_ftype_ext[9:].split('.')
                    if ftype != 'A':
                        continue
                    
                    if length==2:  # most situations
                        to_zfn = zfn
                    else :
                        to_zfn = loc_id + '/' + loc_id_ftype_ext
                        print(fn.split('/')[-1], zfn, to_zfn)

                    if to_zfn not in existing_fns:
                        zf = z.read(zfn)
                        wzf.writestr(to_zfn, zf)
                        existing_fns.add(to_zfn)

                except Exception as e:
                    print(fn, zfn, e)

working on 0th, geography-socioeconomics/2010CensusGeography/Urban/702825582647_s.zip
working on 100th, geography-socioeconomics/2010CensusGeography/Urban/702825583644_s.zip
working on 200th, geography-socioeconomics/2010CensusGeography/Urban/702825584641_s.zip
702825585358_s.zip 090080001A.dbf 090080001/090080001A.dbf
702825585358_s.zip 090080001A.prj 090080001/090080001A.prj
702825585358_s.zip 090080001A.shp 090080001/090080001A.shp
702825585358_s.zip 090080001A.shx 090080001/090080001A.shx
working on 300th, geography-socioeconomics/2010CensusGeography/Urban/702825585648_s.zip
working on 400th, geography-socioeconomics/2010CensusGeography/Urban/702825586645_s.zip
working on 500th, geography-socioeconomics/2010CensusGeography/Urban/702825587642_s.zip
working on 600th, geography-socioeconomics/2010CensusGeography/Urban/702825588649_s.zip
702825589196_s.zip 141250001A.dbf 141250001/141250001A.dbf
702825589196_s.zip 141250001A.prj 141250001/141250001A.prj
702825589196_s.zip 141250001A.sh

#### extract mza urban as one zip

In [10]:
with zipfile.ZipFile(f'{folder}/Urban_Mza.zip', 'a', compression=zipfile.ZIP_DEFLATED) as wzf:
    existing_fns = set(wzf.namelist())

    # loop over each file
    for i, fn in enumerate(urban_fns):
        if i % 500 == 0:
            print(f'working on {i}th, {fn}')

        # open the file
        with zipfile.ZipFile(fn) as z:
            # each file could have multiple loc
            for zfn in z.namelist():
                if zfn.endswith('/'):
                    continue

                try:
                    zfn_split = zfn.split('/')
                    length = len(zfn_split)
                    loc_id_ftype_ext = zfn_split[-1]
                    loc_id = loc_id_ftype_ext[:9]
                    ftype, ext = loc_id_ftype_ext[9:].split('.')
                    if ftype != 'M':
                        continue
                    
                    if length==2:  # most situations
                        to_zfn = zfn
                    else :
                        to_zfn = loc_id + '/' + loc_id_ftype_ext
                        print(fn.split('/')[-1], zfn, to_zfn)

                    if to_zfn not in existing_fns:
                        zf = z.read(zfn)
                        wzf.writestr(to_zfn, zf)
                        existing_fns.add(to_zfn)

                except Exception as e:
                    print(fn, zfn, e)

working on 0th, geography-socioeconomics/2010CensusGeography/Urban/702825582647_s.zip
702825585358_s.zip 090080001M.dbf 090080001/090080001M.dbf
702825585358_s.zip 090080001M.prj 090080001/090080001M.prj
702825585358_s.zip 090080001M.shp 090080001/090080001M.shp
702825585358_s.zip 090080001M.shx 090080001/090080001M.shx
working on 500th, geography-socioeconomics/2010CensusGeography/Urban/702825587642_s.zip
702825589196_s.zip 141250001M.dbf 141250001/141250001M.dbf
702825589196_s.zip 141250001M.prj 141250001/141250001M.prj
702825589196_s.zip 141250001M.shp 141250001/141250001M.shp
702825589196_s.zip 141250001M.shx 141250001/141250001M.shx
702825589455_s.zip 150260008/150250001/150250001M.dbf 150250001/150250001M.dbf
702825589455_s.zip 150260008/150250001/150250001M.prj 150250001/150250001M.prj
702825589455_s.zip 150260008/150250001/150250001M.shp 150250001/150250001M.shp
702825589455_s.zip 150260008/150250001/150250001M.shx 150250001/150250001M.shx
702825589455_s.zip 150260008/150250002

#### rural

In [14]:
rural_fns = glob.glob(f'{folder}/Rural/*.zip')

In [34]:
with zipfile.ZipFile(f'{folder}/Rural_Loc.zip', 'a', compression=zipfile.ZIP_DEFLATED) as wzf:
    existing_fns = set(wzf.namelist())

    # loop over each file
    for i, fn in enumerate(rural_fns):
        if i % 500 == 0:
            print(f'working on {i}th, {fn}')

        # open the file
        with zipfile.ZipFile(fn) as z:
            # each file could have multiple loc
            for zfn in z.namelist():
                if zfn.endswith('/'):
                    continue

                try:
                    zfn_split = zfn.split('/')
                    length = len(zfn_split)
                    loc_id_ftype_ext = zfn_split[-1]
                    loc_id = loc_id_ftype_ext[:9]
                    ftype, ext = loc_id_ftype_ext[9:].split('.')
                    if ftype != 'L':
                        continue
                    
                    if length==2:  # most situations
                        to_zfn = zfn
                    else :
                        to_zfn = loc_id + '/' + loc_id_ftype_ext
                        print(fn.split('/')[-1], zfn, to_zfn)

                    if to_zfn not in existing_fns:
                        zf = z.read(zfn)
                        wzf.writestr(to_zfn, zf)
                        existing_fns.add(to_zfn)

                except Exception as e:
                    print(fn, zfn, e)

working on 0th, geography-socioeconomics/2010CensusGeography/Rural/702825558086_s.zip
working on 500th, geography-socioeconomics/2010CensusGeography/Rural/702825563226_s.zip
working on 1000th, geography-socioeconomics/2010CensusGeography/Rural/702825568382_s.zip
working on 1500th, geography-socioeconomics/2010CensusGeography/Rural/702825574833_s.zip
working on 2000th, geography-socioeconomics/2010CensusGeography/Rural/702825580391_s.zip


In [35]:
with zipfile.ZipFile(f'{folder}/Rural_Mza.zip', 'a', compression=zipfile.ZIP_DEFLATED) as wzf:
    existing_fns = set(wzf.namelist())

    # loop over each file
    for i, fn in enumerate(rural_fns):
        if i % 500 == 0:
            print(f'working on {i}th, {fn}')

        # open the file
        with zipfile.ZipFile(fn) as z:
            # each file could have multiple loc
            for zfn in z.namelist():
                if zfn.endswith('/'):
                    continue

                try:
                    zfn_split = zfn.split('/')
                    length = len(zfn_split)
                    loc_id_ftype_ext = zfn_split[-1]
                    loc_id = loc_id_ftype_ext[:9]
                    ftype, ext = loc_id_ftype_ext[9:].split('.')
                    if ftype != 'M':
                        continue
                    
                    if length==2:  # most situations
                        to_zfn = zfn
                    else :
                        to_zfn = loc_id + '/' + loc_id_ftype_ext
                        print(fn.split('/')[-1], zfn, to_zfn)

                    if to_zfn not in existing_fns:
                        zf = z.read(zfn)
                        wzf.writestr(to_zfn, zf)
                        existing_fns.add(to_zfn)

                except Exception as e:
                    print(fn, zfn, e)

working on 0th, geography-socioeconomics/2010CensusGeography/Rural/702825558086_s.zip
working on 500th, geography-socioeconomics/2010CensusGeography/Rural/702825563226_s.zip
working on 1000th, geography-socioeconomics/2010CensusGeography/Rural/702825568382_s.zip
working on 1500th, geography-socioeconomics/2010CensusGeography/Rural/702825574833_s.zip
working on 2000th, geography-socioeconomics/2010CensusGeography/Rural/702825580391_s.zip
